## Necessary imports

In [1]:
!pip install -U scikit-learn
# !pip install bokeh

!pip install tf-models-official
!pip install -U tensorflow-addons==0.20.0
# !pip install efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 53.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 22.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 10.8 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/5

In [2]:
import math, re, os, sys
import numpy as np
import pandas as pd
from itertools import islice
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm

# from bokeh.plotting import figure, show, save
# from bokeh.models import HoverTool, LinearColorMapper, ColumnDataSource
# from bokeh.io import output_notebook
# from bokeh.transform import linear_cmap

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, callbacks
from tensorflow_models.vision import augment
# import efficientnet.tfkeras as efficientnet

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score

print("Tensorflow version " + tf.__version__)

D0614 19:29:52.461349638      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0614 19:29:52.461396560      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0614 19:29:52.461400305      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0614 19:29:52.461402988      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0614 19:29:52.461405116      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0614 19:29:52.461407520      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0614 19:29:52.461410312      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0614 19:29:52.

Tensorflow version 2.12.0


In [3]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    if 'GPU' in tf.test.gpu_device_name():
        print('Running on GPU', tf.test.gpu_device_name())
    else:
        print('Running on CPU')

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
elif 'GPU' in tf.test.gpu_device_name():
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: local


INFO:tensorflow:Initializing the TPU system: local


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [4]:
from shutil import rmtree

try:
    rmtree("/kaggle/working/Neural-Networks-2023-Project")
    print("Removing previous project folder")
    del sys.modules['utils.DataLoad']
    del DataLoad
    del sys.modules['utils.DataVisualization']
except:
    print('No previous project folder found in working directory.')

! git clone https://github.com/m-tarro/Neural-Networks-2023-Project.git

sys.path.append('/kaggle/working/Neural-Networks-2023-Project/')

from utils.DataLoad import DataLoad
from utils.DataVisualization import *

No previous project folder found in working directory.
Cloning into 'Neural-Networks-2023-Project'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 219 (delta 8), reused 0 (delta 0), pack-reused 198
Receiving objects: 100% (219/219), 38.71 MiB | 46.96 MiB/s, done.
Resolving deltas: 100% (100/100), done.


## Data exploration

In [5]:
class_counts = {}

image_size = 512
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

data_load = DataLoad(image_size=image_size, batch_size=BATCH_SIZE)
ds_explore = data_load.get_training_dataset(ordered=True, onehot=False, split=False)

# Get the total number of iterations
total_iterations = data_load.NUM_TRAINING_IMAGES

# Use tqdm to create a progress bar
for _, label in tqdm(ds_explore.unbatch(), total=total_iterations, desc='Processing images'):
    i = label.numpy()
    if i not in class_counts:
        # print(data_load.CLASSES[i])
        class_counts[i] = 1
    else:
        class_counts[i] += 1
        
    total_iterations -= 1  # Decrement the total_iterations count

    if total_iterations == 0:
        break  # Exit the loop when all elements have been processed

Processing images: 100%|█████████▉| 12752/12753 [00:14<00:00, 852.56it/s] 


In [6]:
def get_weight_for_class(class_id):
    counting = class_counts[class_id]
    weight = 1 / counting
    return weight

#This is the dictionary to use in model fitting to further tweak it
weight_per_class = {class_id: get_weight_for_class(class_id) for class_id in class_counts.keys()}

#In order to use it, add 
#class_weight = weight_per_class 
#inside the fit function

## Model to implement

The data will load to data_load based on chosen `IMAGE_SIZE` and `BATCH_SIZE`. Then the model has to be compiled within `strategy.scope()`, compiled with chosen `optimizer`, `loss`, and `metrics`.

In [7]:
# Specify the maximum amount of cropping and rotation (sampled randomly)
max_crop = 0.6
max_rotate = 20
AUTO = tf.data.experimental.AUTOTUNE

# Function sampling random degrees for image rotation
def rand_degree():
    lower = -max_rotate * (np.pi/180.0) # degrees -> radian
    upper =  max_rotate * (np.pi/180.0) 
    return np.random.uniform(lower, upper), max_rotate

# Our manual augmentation function
def manual_augment(image, label):
    #data augmentation to prevent overfitting and to find more patterns.

    # Image dimensions
    height, width = image.shape[-3:-1]

    # Crop image
    crop = np.random.uniform(max_crop,1) # Sample crop size
    box_top  = int(np.random.uniform(0,(1-crop)*height)) # Sample crop location
    box_left = int(np.random.uniform(0,(1-crop)*width))

    cropped = tf.image.crop_to_bounding_box(image, box_top, box_left, int(height*crop), int(width*crop)) # Crop image
    image = tf.image.resize(cropped, (height, width), method='bilinear') # Resize to original image size

    # Alter image
    image = tf.image.random_flip_left_right(image)  # Flipping left-right makes sense due to flower variation
    image = tf.image.random_saturation(image, 0, 3) # Random saturation makes sense due to growth cycles, lighting

    # Rotate image
    degree, max_rotate = rand_degree() # Sample angle

    if np.abs(degree) > (max_rotate/6) * (np.pi/180.0): # If angle is big enough, rotate
        image = tfa.image.rotate(image, degree, fill_mode='nearest')  # Rotation makes sense due to flower variation, foto angle


    #image = tf.image.random_flip_up_down(image)    # Flipping up-down does not make sense because flowers don't grow that way
    #image = tf.image.random_brightness(image, 0.1) 

    return image, label

def cutmixup__(image, label):

    CutMixUp = augment.MixupAndCutmix(
        mixup_alpha = 0.8,
        cutmix_alpha = 0.5, # default 1.0
        prob = 0.6, # default 1.0
        switch_prob = 0.5,
        label_smoothing = 0.1,
        num_classes = 104
    )
    cutmix_images, cutmix_labels = CutMixUp(images=image, labels=label)
        
    image2 = tf.reshape(tf.stack(cutmix_images),(BATCH_SIZE,image_size,image_size,3))
    label2 = tf.reshape(tf.stack(cutmix_labels),(BATCH_SIZE,len(data_load.CLASSES)))
    return image2,label2

def cutmixup(batch_inputs: tf.data.Dataset, onehot=True):

    CutMixUp = augment.MixupAndCutmix(
        mixup_alpha = 0.8,
        cutmix_alpha = 0.5, # default 1.0
        prob = 0.6, # default 1.0
        switch_prob = 0.5,
        label_smoothing = 0.1,
        num_classes = 104
    )
    cutmix_images, cutmix_labels = CutMixUp(images=batch_inputs[0], labels=batch_inputs[1])
    if not onehot:
        cutmix_labels = tf.argmax(cutmix_labels, axis=-1)
    
    return cutmix_images, cutmix_labels

def mixup(batch_inputs: tf.data.Dataset, onehot=True):

    CutMixUp = augment.MixupAndCutmix(
        mixup_alpha = 0.8,
        cutmix_alpha = 0.0, # disable CutMix
        prob = 0.6, # default 1.0
        switch_prob = 0.0, # disable CutMix
        label_smoothing = 0.1,
        num_classes = 104
    )
    cutmix_images, cutmix_labels = CutMixUp(images=batch_inputs[0], labels=batch_inputs[1])
    if not onehot:
        cutmix_labels = tf.argmax(cutmix_labels, axis=-1)
    
    return cutmix_images, cutmix_labels

def cutmix(batch_inputs: tf.data.Dataset, onehot=True):

    CutMixUp = augment.MixupAndCutmix(
        mixup_alpha = 0.0, # disable MixUp
        cutmix_alpha = 0.5, # default 1.0
        prob = 0.6, # default 1.0
        switch_prob = 0.5,
        label_smoothing = 0.1,
        num_classes = 104
    )
    cutmix_images, cutmix_labels = CutMixUp(images=batch_inputs[0], labels=batch_inputs[1])
    if not onehot:
        cutmix_labels = tf.argmax(cutmix_labels, axis=-1)
    
    return cutmix_images, cutmix_labels

In [8]:
IMAGE_SIZE = [image_size, image_size]
EPOCHS = 30
STEPS_PER_EPOCH = data_load.TRAINING_STEPS_PER_EPOCH

model_name = 'combination_densenet201'
onehot = True

ds_train = data_load.get_training_dataset(image_augment=manual_augment, batch_augment=cutmixup__, onehot=onehot, split=False)
ds_valid = data_load.get_validation_dataset(onehot=onehot)
ds_test = data_load.get_test_dataset(ordered=True)

Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


### Learning rate

In [9]:
# define a fine-tuned schedule for the Learning Rate Scheduler 
def exponential_lr(epoch,
                  start_lr=0.00001,min_lr=0.00001,max_lr=0.00005,
                  rampup_epochs = 5, sustain_epochs = 0,
                  exp_decay = 0.8):  # original exp_decay = 0.8
    def lr(epoch, start_lr, min_lr,max_lr,rampup_epochs,sustain_epochs,
          exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr= ((max_lr-start_lr)/
                rampup_epochs * epoch + start_lr)
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr 
        else:
            lr = ((max_lr - min_lr)* exp_decay ** (epoch-rampup_epochs-sustain_epochs)
                  + min_lr)
            
        return lr
    return lr(epoch,start_lr,min_lr,max_lr,rampup_epochs,sustain_epochs,exp_decay)

# set learning rate scheduler for callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(schedule=exponential_lr,verbose=True)

# learning rate chart
# epoch_rng = [i for i in range(EPOCHS)] 
# y = [exponential_lr(x) for x in epoch_rng]
# plt.plot(epoch_rng,y)
# plt.xlim(-1, EPOCHS)

# print("Learning rate schedule: start = {:.3g}; peak = {:.3g}; end = {:.3g}".format(y[0], max(y), y[-1]))

### Model compiling

In [10]:
with strategy.scope():

#     pretrained_model = efficientnet.EfficientNetB7(
#              weights = 'noisy-student', 
#              include_top = False,
#              input_shape = [*IMAGE_SIZE, 3])

    pretrained_model = tf.keras.applications.DenseNet201(
             weights = 'imagenet', 
             include_top = False,
             input_shape = [*IMAGE_SIZE, 3])

#     pretrained_model = tf.keras.applications.xception.Xception(
#              weights = 'imagenet',
#              include_top = False ,
#              input_shape = [*IMAGE_SIZE, 3])

    pretrained_model.trainable = True
    
    model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(data_load.CLASSES), activation='softmax')
    ])

74836368/74836368 [==============================] - 1s 0us/step


In [13]:
loss_measure = '' if onehot==True else 'sparse_'
# sparse measures used when onehot is not applied to save resources

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=5, # how many epochs to wait before stopping
    restore_best_weights=True,
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet201 (Functional)    (None, 16, 16, 1920)      18321984  
                                                                 
 global_average_pooling2d (G  (None, 1920)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 104)               199784    
                                                                 
Total params: 18,521,768
Trainable params: 18,292,712
Non-trainable params: 229,056
_________________________________________________________________


In [14]:
history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback, early_stopping],
    class_weight = weight_per_class #tuning11
)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30


2023-06-14 19:46:30.476586: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-14 19:46:33.410962: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


99/99 [==============================] - ETA: 0s - loss: 0.0201 - categorical_accuracy: 0.5874

2023-06-14 19:48:30.301972: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-06-14 19:48:30.880074: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


99/99 [==============================] - 221s 1s/step - loss: 0.0201 - categorical_accuracy: 0.5874 - val_loss: 2.3362 - val_categorical_accuracy: 0.4275 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0008100000379979611.
Epoch 2/30
99/99 [==============================] - 98s 997ms/step - loss: 0.0177 - categorical_accuracy: 0.6611 - val_loss: 1.5551 - val_categorical_accuracy: 0.6371 - lr: 8.1000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0005060000335611404.
Epoch 3/30
99/99 [==============================] - 93s 939ms/step - loss: 0.0157 - categorical_accuracy: 0.7415 - val_loss: 0.8543 - val_categorical_accuracy: 0.8192 - lr: 5.0600e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.00023240000449121004.
Epoch 4/30
99/99 [==============================] - 94s 951ms/step - loss: 0.0155 - categorical_accuracy: 0.7522 - val_loss: 0.6914 - val_categorical_accuracy: 0.8599 - lr: 2.3240e-04

Epoch 5: LearningRateScheduler setting 

KeyboardInterrupt: 

In [ ]:
%matplotlib inline

display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211
)
display_training_curves(
    history.history['categorical_accuracy'],
    history.history['val_categorical_accuracy'],
    'accuracy',
    212
)

In [ ]:
np.save(f'history_{model_name}.npy',history.history)
model.save(f'model_{model_name}.h5')

## Confusion matrix

In [ ]:
# print(f'Best model based on F1 score is {best_f1_model}')

# Load validation dataset from DataLoad object
cmdataset = data_load.get_validation_dataset(ordered=True, onehot=False)
images_ds = cmdataset.map(lambda image, label: image)
labels_ds = cmdataset.map(lambda image, label: label).unbatch()

# Get correct labels and model predictions
cm_correct_labels = next(iter(labels_ds.batch(data_load.NUM_TRAINING_IMAGES))).numpy()

cm_probabilities = model.predict(images_ds)
cm_predictions = np.argmax(cm_probabilities, axis=-1)

# Compute confusion matrix and normalize
labels = range(len(data_load.CLASSES))
cmat = confusion_matrix(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
)
cmat = (cmat.T / cmat.sum(axis=1)).T


In [ ]:
score = f1_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
precision = precision_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)
recall = recall_score(
    cm_correct_labels,
    cm_predictions,
    labels=labels,
    average='macro',
)

display_confusion_matrix(cmat, score, precision, recall)

## Visual validation

In [ ]:
dataset = cmdataset.unbatch().batch(20)
batch = iter(dataset)

In [ ]:
images, labels = next(batch)
probabilities = model.predict(images)
predictions = np.argmax(probabilities, axis=-1)
display_batch_of_images((images, labels), predictions)

# Predictions to submit

In [ ]:
def gen_submission_csv(model, data_load=DataLoad(), model_name=None, verbose=False):
    if not model_name:
        print('This will generate a final submission.csv file')
        filename = 'submission.csv'
    else:
        filename = f'submission_{model_name}.csv'
    
    print('Computing predictions...')
    ds_test = data_load.get_test_dataset(ordered=True)
    test_images_ds = ds_test.map(lambda image, idnum: image).batch(BATCH_SIZE)
    probabilities = model.predict(test_images_ds, steps=data_load.TEST_STEPS_PER_EPOCH+1)
    predictions = np.argmax(probabilities, axis=-1)
    
    print(f'Generating {filename} file...')
    test_ids_ds = ds_test.map(lambda image, idnum: idnum)
    test_ids = next(iter(test_ids_ds.batch(data_load.NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
    
    np.savetxt(filename, 
               np.rec.fromarrays([test_ids, predictions]), 
               fmt=['%s', '%d'], 
               delimiter=',', 
               header='id,label', 
               comments='')
    if verbose:
        print()
        !head submission{model_name}.csv

In [ ]:
# If submitting to the competition, model_name should be either ignored or None

gen_submission_csv(model, data_load, model_name=None)

# Submitting

If you haven't already, create your own editable copy of this notebook by clicking on the Copy and Edit button in the top right corner. Then, submit to the competition by following these steps:

1. Begin by clicking on the blue Save Version button in the top right corner of the window. This will generate a pop-up window.
2. Ensure that the Save and Run All option is selected, and then click on the blue Save button.
3. This generates a window in the bottom left corner of the notebook. After it has finished running, click on the number to the right of the Save Version button. This pulls up a list of versions on the right of the screen. Click on the ellipsis (...) to the right of the most recent version, and select Open in Viewer. This brings you into view mode of the same page. You will need to scroll down to get back to these instructions.
4. Click on the Output tab on the right of the screen. Then, click on the file you would like to submit, and click on the blue Submit button to submit your results to the leaderboard.

You have now successfully submitted to the competition!